In [30]:
from src.model.filterData import filterData
from src.model.riskMatrix import riskMatrix
from src.model.calculateTransformerInformation import calculateTransformerInformation

import pandas as pd
from dotenv import load_dotenv
from fastapi import FastAPI
import pyodbc
import os

load_dotenv('.env')

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
driver = os.getenv('DRIVER')

connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=Yes"
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()


In [31]:
codigo_operacional = "INAT7-01"
familia = "Autotransformador"
resultado = riskMatrix(cursor, codigo_operacional, familia).processRiskMatrix()

In [42]:
colunas = ['CodigoCalculo', 'ResultadoCalculo', 'UltimaAtualizacaoCalculo', 'descricaoInstalacaoEletrica', 'descricaoEquipamento', 'VariavelCodigo', 'ValorVariavel', 'Abreviatura', 'Nome']
dados = [dict(zip(colunas, row)) for row in resultado]
df = pd.DataFrame(dados)

print(df)

       CodigoCalculo  ResultadoCalculo UltimaAtualizacaoCalculo  \
0             264836               146                  DEN_ATV   
1             264836               145                  NUM_ATV   
2             264836               153                  DEN_BCH   
3             264836               161                  DEN_CMT   
4             264836               160                  NUM_CMT   
...              ...               ...                      ...   
107563        264845               181             MON_MULTIGAS   
107564        264845               181             MON_MULTIGAS   
107565        264845               181             MON_MULTIGAS   
107566        264845               181             MON_MULTIGAS   
107567        264845               181             MON_MULTIGAS   

        VariavelCodigo           ValorVariavel  Abreviatura        Nome  
0                 31.0 2023-05-30 02:04:37.980           13  Itacaiúnas  
1                 22.0 2023-05-30 02:04:37.980 

In [34]:
resultado_df = pd.DataFrame(resultado)
print(resultado_df.columns)



RangeIndex(start=0, stop=1, step=1)
